<a target="_blank" href="https://colab.research.google.com/github/gox6/colab-demos/blob/main/rags/evaluate-rags-rigorously-or-perish.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [60]:
# Installing Python packages & hiding
!pip install --quiet \
  chromadb \
  langchain \
  polars \
  pytube \
  ragas xmltodict \
  youtube-transcript-api

In [91]:
# Importing the packages
from collections import Counter, defaultdict
import os

import chromadb
from langchain_core.documents.base import Document
from langchain_community.document_loaders import YoutubeLoader
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import polars as pl
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context

In [100]:
# Managing secrets
# - If using Colab please use Colab Secrets
# - If running outside Colab please provide secrets as environmental variables

COLAB = os.getenv("COLAB_RELEASE_TAG") is not None

if COLAB:
  from google.colab import userdata, data_table
  # Secrets
  OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
  os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
  runtime_info = "Colab runtime"

  # Enabling Colab's data formatter for pandas
  data_table.enable_dataframe_formatter()
else:
  # Secrets
  OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
  runtime_info = "Non Colab runtime"

print(runtime_info)

Colab runtime


In [81]:
# 30 Youtube videos from BBC News

youtube_bbc_news_videos = ["https://youtu.be/YEHA-u8b43A", "https://youtu.be/TlFfHjOMSXQ", "https://youtu.be/-kZolk9EoMA", "https://youtu.be/JQMZkrz6X08",
                           "https://youtu.be/D1iMZaLjBU4", "https://youtu.be/Sqcv9lCADxE", "https://youtu.be/Sshl0SFO4ZI", "https://youtu.be/mpOyMOZWEcU",
                           "https://youtu.be/Cg3YMWcjLI4", "https://youtu.be/yQIMSv9Luw4", "https://youtu.be/5LYGnqoCLGk", "https://youtu.be/KCepbsLLUMY",
                           "https://youtu.be/lHggWT2iLdo", "https://youtu.be/TTUsxD62398", "https://youtu.be/3YMleRGjeqE", "https://youtu.be/h-5dqQMZTZQ",
                           "https://youtu.be/4GBcZJpw8yI", "https://youtu.be/QOUGlWEpwL4", "https://youtu.be/T05I-SBhXoI", "https://youtu.be/Iz-XY6XfXjk",
                           "https://youtu.be/yyLFQrb--pw", "https://youtu.be/wMqJbMPNM6A", "https://youtu.be/kOcuwLBPBP8", "https://youtu.be/7goRcrFKs3U",
                           "https://youtu.be/FoBOSLofM3E", "https://youtu.be/97nEBjiQI1M", "https://youtu.be/bjmK4lGKNqY", "https://youtu.be/XWLA5A6bpwE",
                           "https://youtu.be/DkmrhVpCmac", "https://youtu.be/Dxar1d1aTUo"
                          ]


count = Counter(youtube_bbc_news_videos)
assert len(count) == 30

In [101]:
# Collecting transcripts of Youtube videos
# Long running cell: around 60s

def get_youtube_transcripts(urls: list[str]) -> list[Document]:

  docs = list(map(lambda url: YoutubeLoader.from_youtube_url(url, add_video_info=True).load()[0], urls))

  return docs

bbc_news = get_youtube_transcripts(youtube_bbc_news_videos)

In [151]:
# Transforming list of LangChain documents into dataframe to review data conveniently

def to_pl(list_of_docs: list[Document]) -> pl.DataFrame:
  data = defaultdict(list)

  for doc in list_of_docs:
    doc_dict = doc.dict()
    for key in doc_dict.keys():
      if key != 'metadata':
        data[key].append(doc_dict[key])
      else:
        metadata = doc_dict['metadata']
        for subkey in metadata.keys():
          data[subkey].append(metadata[subkey])

  df_pl = pl.DataFrame(data)
  df_pl = df_pl.with_columns([(pl.lit('https://youtu.be/') + pl.col('source')).alias('video_url'),
                              (pl.col('page_content').str.split(by=' ').len()).alias("words_no")])

  # df_pl = df_pl.select([ 'publish_date', 'author', 'title',  'video_url', 'view_count', 'length', 'words_no', 'page_content',
  #      ])
  return df_pl


df_pl = to_pl(bbc_news)
df_pd = df_pl.to_pandas()   # Switching to Pandas from Polars dataframe as it is then better displayed in Colab

# Displaying data with Colab formatter, or just in pandas
df_pd

,page_content,source,title,description,view_count,thumbnail_url,publish_date,length,author,type,video_url,words_no
0,well the Tesla now and a whistle blow whistleb...,YEHA-u8b43A,Tesla whistleblower says she wants an Elon Mus...,Unknown,48723,https://i.ytimg.com/vi/YEHA-u8b43A/hq720.jpg,2024-04-16 00:00:00,218,BBC News,Document,https://youtu.be/YEHA-u8b43A,30
1,smoking is the single biggest cause of prevent...,TlFfHjOMSXQ,UK smoking ban: MPs to vote on banning young p...,Unknown,26558,https://i.ytimg.com/vi/TlFfHjOMSXQ/hq720.jpg,2024-04-16 00:00:00,192,BBC News,Document,https://youtu.be/TlFfHjOMSXQ,30
2,the organizers of a conference in Brussels whi...,-kZolk9EoMA,Right-wing event in Brussels told to shut down...,Unknown,24412,https://i.ytimg.com/vi/-kZolk9EoMA/hq720.jpg,2024-04-16 00:00:00,181,BBC News,Document,https://youtu.be/-kZolk9EoMA,30
3,and we begin with events in the Middle East wh...,JQMZkrz6X08,Israel demands sanctions on Iranian missile pr...,Unknown,150330,https://i.ytimg.com/vi/JQMZkrz6X08/hq720.jpg,2024-04-16 00:00:00,458,BBC News,Document,https://youtu.be/JQMZkrz6X08,30
4,now I want to take you back to Greece and to O...,D1iMZaLjBU4,Olympic flame lit in Greece's ancient Olympia ...,Unknown,40224,https://i.ytimg.com/vi/D1iMZaLjBU4/hq720.jpg,2024-04-16 00:00:00,695,BBC News,Document,https://youtu.be/D1iMZaLjBU4,30
5,you're watching the context it's time for our ...,Sqcv9lCADxE,Elon Musk’s AI chatbot generated disinformatio...,Unknown,55637,https://i.ytimg.com/vi/Sqcv9lCADxE/hq720.jpg,2024-04-11 00:00:00,895,BBC News,Document,https://youtu.be/Sqcv9lCADxE,30
6,let's turn to Taiwan rescue efforts are underw...,Sshl0SFO4ZI,Taiwan earthquake: More than 600 stranded a da...,Unknown,159263,https://i.ytimg.com/vi/Sshl0SFO4ZI/hq720.jpg,2024-04-04 00:00:00,445,BBC News,Document,https://youtu.be/Sshl0SFO4ZI,30
7,to Uganda next and the country's constitutiona...,mpOyMOZWEcU,Uganda’s top appeals court rejects petition to...,Unknown,31330,https://i.ytimg.com/vi/mpOyMOZWEcU/hq720.jpg,2024-04-03 00:00:00,219,BBC News,Document,https://youtu.be/mpOyMOZWEcU,30
8,hello I'm Nikki schill welcome to the program ...,Cg3YMWcjLI4,US President Joe Biden vows 'ironclad' support...,Unknown,111440,https://i.ytimg.com/vi/Cg3YMWcjLI4/hq720.jpg,2024-04-11 00:00:00,353,BBC News,Document,https://youtu.be/Cg3YMWcjLI4,30
9,right we're going to look now at UK US diploma...,yQIMSv9Luw4,Lord Cameron meets Antony Blinken after 'priva...,Unknown,44282,https://i.ytimg.com/vi/yQIMSv9Luw4/hq720.jpg,2024-04-09 00:00:00,484,BBC News,Document,https://youtu.be/yQIMSv9Luw4,30


In [119]:
bbc_news

[Document(page_content="well the Tesla now and a whistle blow whistleblower who's battled Elon Musk and the car maker through the courts for a decade has been speaking to the BBC Christina Balon says she's still seeking a public apology for how she was treated after raising safety concerns about Tesla vehicles until 2014 Miss Balon was a rising star at the car maker in the US she spoke to our technology editor Zoe Kleinman I was the only woman in the team like engineering them and uh in the beginning was great in the early days Christina Balon was doing so well at Tesla her initials were engraved on the car's batteries but she says it wasn't long before things took a turn for the worse everything went South when I realized that they were hiding some critical safety issues she claims they pretended that they didn't know and you decided to go right to the top you went to Elon Musk himself I tried to and what exactly was the safety concern they realized that they make a design and gening 

In [ ]:
client = chromadb.Client()


In [140]:
base_docs = bbc_news[20:22]

# generator with openai models
llm = ChatOpenAI(model="gpt-3.5-turbo")
generator_llm = llm
critic_llm = llm
embeddings = OpenAIEmbeddings()

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

# Change resulting question type distribution
distributions = {
    simple: 1,
}


from ragas.testset.docstore import InMemoryDocumentStore



# use generator.generate_with_llamaindex_docs if you use llama-index as document loader
testset = generator.generate_with_langchain_docs(base_docs, 2, distributions)
testset.to_pandas()

embedding nodes:   0%|          | 0/4 [00:00<?, ?it/s]

Generating:   0%|          | 0/2 [00:00<?, ?it/s]

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What was the landmarked judgment at the Europe...,[a landmarked judgment at the European Court o...,Switzerland violated the human rights of a gro...,simple,"[{'source': 'wMqJbMPNM6A', 'title': 'European ...",True
1,What was the landmarked judgment at the Europe...,[a landmarked judgment at the European Court o...,Switzerland violated the human rights of a gro...,simple,"[{'source': 'wMqJbMPNM6A', 'title': 'European ...",True


In [121]:
from langchain.document_loaders import PubMedLoader

loader = PubMedLoader("liver", load_max_docs=10)
documents = loader.load()

Too Many Requests, waiting for 0.20 seconds...
Too Many Requests, waiting for 0.40 seconds...
Too Many Requests, waiting for 0.80 seconds...
Too Many Requests, waiting for 1.60 seconds...


In [138]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# documents = load your documents

# generator with openai models
generator_llm = ChatOpenAI(model="gpt-3.5-turbo-16k")
critic_llm = generator_llm
# critic_llm = ChatOpenAI(model="gpt-4", project_id='proj_pZxIXWAo4jloCvpTcjFmCMk4')
embeddings = OpenAIEmbeddings()

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

# Change resulting question type distribution
distributions = {
    simple: 0.5,
    multi_context: 0.4,
    reasoning: 0.1
}

# use generator.generate_with_llamaindex_docs if you use llama-index as document loader
testset = generator.generate_with_langchain_docs(documents, 1, distributions)
testset.to_pandas()


embedding nodes:   0%|          | 0/20 [00:00<?, ?it/s]

Generating:   0%|          | 0/1 [00:00<?, ?it/s]

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,How is dysregulated piRNA expression associate...,[INTRODUCTION: This article examines the poten...,Dysregulated piRNA expression is associated wi...,simple,"[{'uid': '38627675', 'Title': 'The role of piR...",True
